### TODO
DONE convert API key to a environmental variable
- create header / tailer / blocks
- find meetings with overlapping locations and put in a little jitter to distinguish them on the map
- That means that you collect all the info in a list of dictionaries first, then you go back through them and mess with them.
- output to a file
- put in the README how to make environmental variable for API token and a little about setting up selenium
- README: importing to google mymaps
- push to repo without data (put data file output into .gitignore)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import datetime
import os
from collections import defaultdict
from googlemaps import Client as GoogleMaps
import random

LATLON_SMEAR = 1.e-4
# making the token an environmental variable
# echo GOOGLE_API_KEY=APIT0k3nfR0my0Ur9009l3d3V3L0P3rACc0Un7 >.env
# put this in gitignore; test it.

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY', 'Token Not found')
gmaps = GoogleMaps(GOOGLE_API_KEY)

driver = webdriver.Chrome()
url = 'https://al-anon.info/MtgSearch'
driver.get(url)

select = Select(driver.find_element_by_xpath("//select[@name='ddlStateProvince']"))
select.select_by_value('WA')

driver.implicitly_wait(3)
driver.find_element_by_name('btnSearch').click()

In [2]:
raw_meetings = []
gridrows = driver.find_elements_by_class_name('GridRowMeeting')
gridrows+=driver.find_elements_by_class_name('GridAltRowMeeting')

for i,gridrow in enumerate(gridrows):
    this_dict = {}
    row = gridrow.find_elements_by_tag_name('td')
    print(i,' ',79*'-')
    meeting_time = row[0].text
    meeting_address = ', '.join(row[1].text.split('\n')[1:-1])
    meeting_location = row[1].text.split('\n')[0]
    meeting_notes = row[2].text
    meeting_name = meeting_notes.split('\n')[0]
    group_id = meeting_notes.split('\n')[-1].split(' ')[-1]
    meeting_notes2 = '\n'.join(meeting_notes.split('\n')[1:-1])
    print(f'Meeting:  {meeting_name}')
    print(f'Time:     {meeting_time}')
    print(f'Group id: {group_id}')
    print(f'Location: {meeting_location}')
    print(f'Address:  {meeting_address}')
    print(f'{meeting_notes2}')
    # get lat,lon from google API
    lat,lon = (36.7845921,-76.0076847) # Al-Anon HQ as default
    try:
        time.sleep(1.2) #to add Google API timeout delay if needed
        geocode_result = gmaps.geocode(meeting_address)
        x   = geocode_result[0]['geometry']['location']['lat']
        lat = f'{x:.7f}'
        x   = geocode_result[0]['geometry']['location']['lng']
        lon = f'{x:.7f}'
        print(f'Lat, Lon    {lat}    {lon}')
    except IndexError:
        print("Address was wrong...")
    except Exception as e:
        print("Unexpected error occurred.", e )
    this_dict['name'] = meeting_name
    this_dict['time'] = meeting_time
    this_dict['location'] = meeting_location
    this_dict['address'] = meeting_address
    this_dict['notes'] = meeting_notes2
    this_dict['id'] = group_id
    this_dict['lat-lon'] = (lat,lon) 
    raw_meetings.append(this_dict)
        

0   -------------------------------------------------------------------------------
Meeting:  HOPE FOR TODAY AFG
Time:     Monday 07:00 AM
Group id: 502461
Location: St. Paul's United Church
Address:  6512 12th Ave NW, Seattle WA 98117
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    47.6762040    -122.3716002
1   -------------------------------------------------------------------------------
Meeting:  TGIM AFG
Time:     Monday 10:00 AM
Group id: 38244
Location: First Lutheran Community Church
Address:  2483 Mitchell Rd SE, Port Orchard WA 98366
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    47.5258709    -122.6290130
2   -------------------------------------------------------------------------------
Meeting:  EASY DOES IT AFG
Time:     Monday 10:00 AM
Group id: 26821
Location: St Phillips Episcopal Church
Address:  4312 84th St NE, Marysville WA 98270
Handicap Access & Child Care & Fragrance Free
Families, Friends, and Observers Welcome
Lat, L

Lat, Lon    45.6245844    -122.5798220
24   -------------------------------------------------------------------------------
Meeting:  UPPER VALLEY AFG
Time:     Monday 07:00 PM
Group id: 53655
Location: United Methodist Church
Address:  418 Evans St, Leavenworth WA 98826
Families, Friends, and Observers Welcome
Lat, Lon    47.5972653    -120.6623959
25   -------------------------------------------------------------------------------
Meeting:  MAGNOLIA MONDAY NITE FIRESIDE AFG
Time:     Monday 07:00 PM
Group id: 63177
Location: Magnolia United Church Of Christ
Address:  3555 W McGraw St, Seattle WA 98199
Families, Friends, and Observers Welcome
Lat, Lon    47.6390951    -122.4035156
26   -------------------------------------------------------------------------------
Meeting:  TGIM AFG
Time:     Monday 07:00 PM
Group id: 14167
Location: St Lukes Episcopal Church
Address:  428 King St, Wenatchee WA 98801
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    47.4220887    -1

Lat, Lon    46.7086184    -122.9585258
48   -------------------------------------------------------------------------------
Meeting:  ROCK COVE AFG
Time:     Tuesday 06:00 PM
Group id: 500478
Location: United Methodist Church
Address:  35 NW Jefferson St, Stevenson WA 98648
Families, Friends, and Observers Welcome
Lat, Lon    45.6948087    -121.8860059
49   -------------------------------------------------------------------------------
Meeting:  Happy Our Program Exists AFG
Time:     Tuesday 06:30 PM
Group id: 30715774
Location: Mossyrock Grange
Address:  152 Isbell Road, Mossyrock WA 98564
Families, Friends, and Observers Welcome
Lat, Lon    46.5244869    -122.4914753
50   -------------------------------------------------------------------------------
Meeting:  Just for Today AFG
Time:     Tuesday 06:30 PM
Group id: 51248
Location: St Luke's Episcopal Church
Address:  525 N 5th Ave, Sequim WA 98382
Park in lot behind the church. Enter on South side of the building down the wheelchair 

Lat, Lon    47.4930302    -117.5771686
72   -------------------------------------------------------------------------------
Meeting:  WEDNESDAY AFG LUNCH BREAK
Time:     Wednesday 12:00 PM
Group id: 52824
Location: Dungeness Valley Lutheran Church
Address:  925 N Sequim Ave, Sequim WA 98382
Church parish house
Handicap Access & Fragrance Free
Families, Friends, and Observers Welcome
Lat, Lon    48.0921140    -123.1034581
73   -------------------------------------------------------------------------------
Meeting:  WEDNESDAY NOON NEWPORT AFG
Time:     Wednesday 12:00 PM
Group id: 30558120
Location: American Lutheran Church
Address:  332801 Highway 2, Newport WA 99156
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    48.1734617    -117.0702842
74   -------------------------------------------------------------------------------
Meeting:  MIRACLES IN HILLYARD AFG
Time:     Wednesday 01:30 PM
Group id: 30673637
Location: Saint Peter's Lutheran Church Meeting House
Address

Lat, Lon    45.6153901    -122.5606165
95   -------------------------------------------------------------------------------
Meeting:  SEEDS OF HOPE AFG
Time:     Thursday 12:00 PM
Group id: 30618593
Location: Bayside Community Church
Address:  25992 Barber Cut Off Road NE, Kingston WA 98346
Enter through back parking lot, thru double doors. Follow signs
Families, Friends, and Observers Welcome
Lat, Lon    47.7991096    -122.5082369
96   -------------------------------------------------------------------------------
Meeting:  THURSDAY NOON EAST BREMERTON AFG
Time:     Thursday 12:00 PM
Group id: 62033
Location: Holy Trinity Catholic Church
Address:  4215 Pine Rd NE, Bremerton WA 98310
Educational wing 12
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    47.6039025    -122.6409277
97   -------------------------------------------------------------------------------
Meeting:  THE 3 C's AFG
Time:     Thursday 12:15 PM
Group id: 30518847
Location: Harbor Life Church
Addres

...

Lat, Lon    48.0399789    -122.4119727
235   -------------------------------------------------------------------------------
Meeting:  Gratitude Group AFG
Time:     Wednesday 10:00 AM
Group id: 14412
Location: Beautiful Savior Lutheran Church
Address:  12513 SE Mill Plain Blvd, Vancouver WA 98684
Families, Friends, and Observers Welcome
Lat, Lon    45.6192961    -122.5434756
236   -------------------------------------------------------------------------------
Meeting:  SILVANA SERENITY AFG
Time:     Wednesday 10:30 AM
Group id: 30659155
Location: Peace Lutheran Church
Address:  1717 Ole Larson Rd, Silvana WA 98287
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    48.2026319    -122.2556797
237   -------------------------------------------------------------------------------
Meeting:  DAFFODIL VALLEY AFG
Time:     Wednesday 10:30 AM
Group id: 501515
Location: First Christian Church
Address:  432 Wood Ave, Sumner WA 98390
Handicap Access
Families and Friends Only
Lat, 

Lat, Lon    48.1117859    -123.4440788
259   -------------------------------------------------------------------------------
Meeting:  NEW BEGINNINGS AFG
Time:     Wednesday 07:00 PM
Group id: 27883
Location: St Rose Catholic Church
Address:  701 26th Ave, Longview WA 98632
Father mulligan hall
Families, Friends, and Observers Welcome
Lat, Lon    46.1325717    -122.9542954
260   -------------------------------------------------------------------------------
Meeting:  ONE DAY AT A TIME AFG
Time:     Wednesday 07:00 PM
Group id: 30528516
Location: Stanwood Senior Center
Address:  7430 276th St NW, Stanwood WA 98292
Families, Friends, and Observers Welcome
Lat, Lon    48.2460647    -122.3333491
261   -------------------------------------------------------------------------------
Meeting:  MIDWEEK MEN'S AFG
Time:     Wednesday 07:30 PM
Group id: 53963
Location: Salt House Church
Address:  11920 NE 80th St, Kirkland WA 98033
Handicap Access
Families, Friends, and Observers Welcome
Men Begin

Lat, Lon    47.7339183    -122.6436598
283   -------------------------------------------------------------------------------
Meeting:  HAZEL DELL AFG
Time:     Thursday 07:15 PM
Group id: 14408
Location: Messiah Lutheran Church
Address:  905 NW 94th St, Vancouver WA 98665
Handicap Access
Families and Friends Only
Lat, Lon    45.6891768    -122.6808957
284   -------------------------------------------------------------------------------
Meeting:  STANWOOD AFG
Time:     Thursday 07:30 PM
Group id: 31577
Location: Stanwood United Methodist Church
Address:  27128 102nd Dr. NW, Stanwood WA 98292
Handicap Access & Smoking Permitted
Families, Friends, and Observers Welcome
Lat, Lon    48.2427909    -122.3719440
285   -------------------------------------------------------------------------------
Meeting:  BELLEVUE LAKE HILLS AFG
Time:     Thursday 07:30 PM
Group id: 14118
Location: St Andrews Church
Address:  2650 148th Ave SE, Bellevue WA 98007
Alateen at same location
Handicap Access
Famili

Lat, Lon    47.4664718    -122.3476501
307   -------------------------------------------------------------------------------
Meeting:  TGIF AFG
Time:     Friday 08:00 PM
Group id: 14171
Location: Maplewood Presbyterian Church
Address:  19523 84th Ave W, Edmonds WA 98026
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    47.8220405    -122.3461164
308   -------------------------------------------------------------------------------
Meeting:  Friday Night LGBTQ AFG
Time:     Friday 08:00 PM
Group id: 14299
Location: Peer Seatle
Address:  1520 Bellevue Ave Suite 100, Seattle WA 98112
Handicap Access
Families and Friends Only
LGBT
Lat, Lon    47.6147871    -122.3266933
309   -------------------------------------------------------------------------------
Meeting:  West Seattle Saturday Morning AFG
Time:     Saturday 08:30 AM
Group id: 37671
Location: St. John's Episcopal
Address:  3050 California Ave SW, Seattle WA 98116
Located at the Parish Hall
Handicap Access & Child C

Lat, Lon    48.7562157    -122.4894588
329   -------------------------------------------------------------------------------
Meeting:  Personal Freedom Literature Study AFG
Time:     Sunday 10:00 AM
Group id: 30749747
Location: Olympic Memorial Hospital
Address:  939 Caroline St., Port Angeles WA 98362
Families and Friends Only
Lat, Lon    48.1154826    -123.4144823
330   -------------------------------------------------------------------------------
Meeting:  SUNDAY MORNING AFG
Time:     Sunday 10:30 AM
Group id: 14363
Location: Alano Club
Address:  1700 W 7th Ave, Spokane WA 99204
Handicap Access
Families and Friends Only
Lat, Lon    47.6494963    -117.4373268
331   -------------------------------------------------------------------------------
Meeting:  Sunday Nooners AFG
Time:     Sunday 12:00 PM
Group id: 30608906
Location: Alano Club Of The Eastside
Address:  12302 NE 8th St, Bellevue WA 98005
Handicap Access
Families, Friends, and Observers Welcome
Lat, Lon    47.6177382    -122

In [3]:
# reformat data storage 
meeting_latlon_dict = defaultdict(lambda: defaultdict(list))
for mtg in raw_meetings:
    group_id = mtg['id']
    (lat,lon) = mtg['lat-lon']
    meeting_latlon_dict[(lat,lon)][group_id].append(mtg)

In [4]:
for latlon_key in meeting_latlon_dict.keys():
    print(80*'=')
    print(latlon_key)
    number_of_meetings_at_latlon = len(meeting_latlon_dict[latlon_key].keys())
    for group_id_key in meeting_latlon_dict[latlon_key].keys():
        print(80*'-')
        print(group_id_key)
        print(meeting_latlon_dict[latlon_key][group_id_key])
        number_of_meeting_times = len(meeting_latlon_dict[latlon_key][group_id_key])
        if number_of_meeting_times>1:
            # concatenate group information
            unified_dict = {}
            unified_dict['name'] = meeting_latlon_dict[latlon_key][group_id_key][0]['name']
            unified_dict['time'] = '/'.join([meeting_time['time'] for meeting_time in meeting_latlon_dict[latlon_key][group_id_key]])
            print('replacing times: ',unified_dict['time'])
            unified_dict['location'] = meeting_latlon_dict[latlon_key][group_id_key][0]['location']
            unified_dict['address'] = meeting_latlon_dict[latlon_key][group_id_key][0]['address']
            uniq_notes = set([meeting_time['notes'] for meeting_time in meeting_latlon_dict[latlon_key][group_id_key]])
            notes = []
            for meeting_instance in meeting_latlon_dict[latlon_key][group_id_key]:
                if meeting_instance['notes'] not in notes:
                    notes.append(meeting_instance['notes'])
            unified_dict['notes'] = '\n'.join(notes)
            print('replacing notes: ',unified_dict['notes'])
            unified_dict['id'] = meeting_latlon_dict[latlon_key][group_id_key][0]['id']
            random.seed(group_id_key)
            x   = float(latlon_key[0])+random.gauss(0.,LATLON_SMEAR)
            lat = f'{x:.7f}'
            x   = float(latlon_key[1])+random.gauss(0.,LATLON_SMEAR)
            lon = f'{x:.7f}'
            if number_of_meetings_at_latlon>1:
                unified_dict['lat-lon'] = (lat,lon)
                print('Replacing lat-lon ',(lat,lon))
            else:
                unified_dict['lat-lon'] = meeting_latlon_dict[latlon_key][group_id_key][0]['lat-lon']
            meeting_latlon_dict[latlon_key][group_id_key] = [unified_dict]
        elif number_of_meetings_at_latlon>1:
            random.seed(group_id_key)
            x   = float(latlon_key[0])+random.gauss(0.,LATLON_SMEAR)
            lat = f'{x:.7f}'
            x   = float(latlon_key[1])+random.gauss(0.,LATLON_SMEAR)
            lon = f'{x:.7f}'
            meeting_latlon_dict[latlon_key][group_id_key][0]['lat-lon'] = (lat,lon)
            print('Replacing lat-lon ',(lat,lon))

('47.6762040', '-122.3716002')
--------------------------------------------------------------------------------
502461
[{'name': 'HOPE FOR TODAY AFG', 'time': 'Monday 07:00 AM', 'location': "St. Paul's United Church", 'address': '6512 12th Ave NW, Seattle WA 98117', 'notes': 'Handicap Access\nFamilies, Friends, and Observers Welcome', 'id': '502461', 'lat-lon': ('47.6762040', '-122.3716002')}, {'name': 'HOPE FOR TODAY AFG', 'time': 'Wednesday 07:00 AM', 'location': "St. Paul's United Church", 'address': '6512 12th Ave NW, Seattle WA 98117', 'notes': 'Handicap Access\nFamilies, Friends, and Observers Welcome', 'id': '502461', 'lat-lon': ('47.6762040', '-122.3716002')}, {'name': 'HOPE FOR TODAY AFG', 'time': 'Friday 07:00 AM', 'location': "St. Paul's United Church", 'address': '6512 12th Ave NW, Seattle WA 98117', 'notes': 'Handicap Access\nFamilies, Friends, and Observers Welcome', 'id': '502461', 'lat-lon': ('47.6762040', '-122.3716002')}]
replacing times:  Monday 07:00 AM/Wednesday 07

Replacing lat-lon  ('45.6892225', '-122.6808551')
('47.4022373', '-122.3224371')
--------------------------------------------------------------------------------
65354
[{'name': 'STEPS TO SERENITY AFG', 'time': 'Tuesday 07:00 PM', 'location': 'Des Moines United Methodist Church', 'address': '22225 9th Ave S, Des Moines WA 98198', 'notes': 'Handicap Access\nFamilies, Friends, and Observers Welcome', 'id': '65354', 'lat-lon': ('47.4022373', '-122.3224371')}]
('46.6049427', '-120.5638445')
--------------------------------------------------------------------------------
26066
[{'name': 'NEW LIFE AFG', 'time': 'Tuesday 07:00 PM', 'location': "St Timothy's Episcopal Church", 'address': '4105 Richey Rd, Yakima WA 98908', 'notes': 'Families and Friends Only\nBeginners', 'id': '26066', 'lat-lon': ('46.6049427', '-120.5638445')}]
('47.3091472', '-122.2152519')
--------------------------------------------------------------------------------
30543700
[{'name': 'Parents and Friends AFG', 'time': 'T

14299
[{'name': 'Friday Night LGBTQ AFG', 'time': 'Friday 08:00 PM', 'location': 'Peer Seatle', 'address': '1520 Bellevue Ave Suite 100, Seattle WA 98112', 'notes': 'Handicap Access\nFamilies and Friends Only\nLGBT', 'id': '14299', 'lat-lon': ('47.6147871', '-122.3266933')}]
('47.5758849', '-122.3861912')
--------------------------------------------------------------------------------
37671
[{'name': 'West Seattle Saturday Morning AFG', 'time': 'Saturday 08:30 AM', 'location': "St. John's Episcopal", 'address': '3050 California Ave SW, Seattle WA 98116', 'notes': 'Located at the Parish Hall\nHandicap Access & Child Care\nFamilies, Friends, and Observers Welcome', 'id': '37671', 'lat-lon': ('47.5758849', '-122.3861912')}]
('47.2649529', '-122.4621863')
--------------------------------------------------------------------------------
30620540
[{'name': 'ROCKY BOTTOM AFG', 'time': 'Saturday 09:00 AM', 'location': 'Immanuel Presbyterian Church', 'address': '901 N J St, Tacoma WA 98403', 'no

In [5]:
final_list_of_dicts = []
for latlon_key in meeting_latlon_dict.keys():
    for group_id_key in meeting_latlon_dict[latlon_key].keys():
        final_list_of_dicts.append(meeting_latlon_dict[latlon_key][group_id_key][0])
        
final_list_of_dicts = sorted(final_list_of_dicts, key = lambda i: i['name'])

In [6]:
# reprocess all meeting information:
# DONE DOESN'T WORK) can we use description for color coding?  How is this done in the 
#    source image for colors: https://previews.123rf.com/images/corinapetre/corinapetre1908/corinapetre190800065/130046874-hand-lettered-days-of-the-week-t-shirt-print-or-calendar-card-vector-illustration.jpg
# 6) reorder sensibly
def get_day_color(meeting_time):
    # looks like pin color import is unsupported:
    # https://stackoverflow.com/questions/37687998/kml-markers-have-lost-their-color-but-only-in-the-js-api
    # 
    color_monday = '83cef6'.upper()
    color_tuesday = 'fee68d'.upper()
    color_wednessday = 'bba2ca'.upper()
    color_thursday = 'ef9dc2'.upper()
    color_friday = 'f4b99a'.upper()
    color_saturday = 'a5d2af'.upper()
    color_sunday = 'a5b4dd'.upper()
    color_default = '030303'.upper()

    day = meeting_time.split(' ')[0]
    if (day=='Monday'):
        return(color_monday)
    elif (day=='Tuesday'):
        return(color_tuesday)
    elif (day=='Wednesday'):
        return(color_wednesday)
    elif (day=='Thursday'):
        return(color_thursday)
    elif (day=='Friday'):
        return(color_friday)
    elif (day=='Saturday'):
        return(color_saturday)
    elif (day=='Sunday'):
        return(color_sunday)
    else:
        return(color_default)

In [14]:
# KML code
with open('meeting_location_header.kml', 'r') as f:
    FILE_KML_HEADER = f.read()
with open('meeting_location_main.kml', 'r') as f:
    FILE_KML_MAIN = f.read()
with open('meeting_location_tailer.kml', 'r') as f:
    FILE_KML_TAILER = f.read()

x = datetime.datetime.now()
date_msg = f'{x.strftime("%b")}-{x.day}-{x.year}'

with open('all_meetings.kml','w+') as f:
    f.write(FILE_KML_HEADER.format(date_msg))
    for item in final_list_of_dicts:
        meeting_time = item['time']
        meeting_name = item['name']
        meeting_location = item['location']
        meeting_address = item['address']
        meeting_notes2 = item['notes']
        group_id = item['id']
        lat,lon = item['lat-lon']
        print(lat,lon,group_id,meeting_name)
        info_string = '<br>'.join([meeting_time,meeting_location,meeting_address,meeting_notes2.replace('\n','<br>'),'Group Id:'+group_id])
        color = '0288D1' # get_day_color(meeting_time)
        f.write(FILE_KML_MAIN.format(meeting_name,info_string,color,lon,lat))
    f.write(FILE_KML_TAILER)
f.close()

48.8488217 -122.6380172 30720445 1st Things 1st AFG
47.7560285 -122.2861782 30518096 3RD PLACE LITERATURE STUDY AFG
47.2574215 -122.4451586 30588866 ADULT CHILDREN AFG
47.8221846 -122.3461183 30623421 ALDERWOOD AFG
48.5153915 -122.6179585 51471 ANACORTES AFG
48.5152551 -122.6178940 30609499 ANACORTES MEN'S AFG
48.1964520 -122.1141632 14108 ARLINGTON TUESDAY AFG
48.5075934 -122.2236898 14340 ATTITUDE TO GRATITUDE AFG
47.2942694 -122.2126657 23848 AUBURN FRIDAY MORNING AFG
47.2939655 -122.2230502 14111 AUBURN SUNDAY NIGHT AFG
47.2943198 -122.2127922 14109 AUBURN WEDNESDAY NIGHT AFG
45.6246815 -122.5799444 14404 AWARENESS NEWCOMERS AFG
47.7338348 -122.6435964 502578 BACK TO BASICS POULSBO AFG
47.6310622 -122.5189693 49016 BAINBRIDGE SUNDAY MORNING STEP STUDY AFG
47.6761322 -122.3716436 14300 BALLARD SUNDAY NITE AFG
47.5858055 -122.1421644 14118 BELLEVUE LAKE HILLS AFG
48.9883341 -122.7460959 501896 BLAINE SERENITY AFG
47.8134086 -122.1818408 30631777 BOTHELL MONDAY NIGHT AFG
47.7701196 -1

### Code Graveyard

In [ ]:
from bs4 import BeautifulSoup
import requests
page = requests.get(url)#, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

#x = soup.find('script', type='text/javascript')




In [ ]:
soup